In [24]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pandas as pd
import numpy as np

In [8]:
lines=pd.read_csv("datasets/processed_data.csv",encoding='utf-8')
print(lines['Hindi'].value_counts())
lines.head(20)

धन्यवाद।                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   6
(तालियाँ)                                                                                                                                                                                                                                                                  

,Hindi,Telugu
0,कुछ वर्षों पहले मुझे ऐसा लगा जैसे मैं किसी उदा...,"కొన్ని సంవత్సరాల ముందు, నేను బాగా ఆచరానములో ఉన..."
1,यह विचार बहुत ही सरल है |,ఈ ఆలోచన చాలా సులభమైనది.
2,ऐसे किसी चीज के बारे में सोचिये जिसे आप हमेशा ...,మీ జీవితములో మీరు చేయాలి అనుకునే పనిని ఆలోచించ...
3,"यह पता चला है कि 30 दिन पर्याप्त समय है, कोई न...",ఫలితము దక్కుతుంది. 30 రోజులనేది మీ జీవితములో ఒ...
4,इस 30 दिन की चुनौतियों के दौरान मैंने कुछ बाते...,ఈ 30 రోజులు పాటించే విధానములో నేను కొన్ని విషయ...
5,"पहला था कि, बजाय इसके के महीने यूँ ही भूलते हु...","మొదటిది, నెలలు గడిచే కొద్ది నెలల్లో జరిగే విషయ..."
6,यह उस चुनौती का एक हिस्सा था जिसमे प्रतिदिन मै...,"ఇది కూడా ఆ సవాలులో ఒక భాగము,నేను ఒక నెలకి ఒక ఫ..."
7,और मुझे याद है मैं उस दिन कहाँ था और क्या कर र...,"నేను ఎక్కడున్నానని మరియు, ఆ రోజు ఏమి చేస్తున్న..."
8,मैंने यह भी पाया कि जैसे जैसे मैं 30 दिनों की ...,నేను గమనించింది ఏమిటి అంటే 30 రోజుల సవాళ్ళు చే...
9,मैं एक कंप्यूटर पर काम करके कुर्सियां तोड़ने व...,నేను కంప్యూటర్ కార్యాలయంలో కంప్యూటర్ గురించి మ...


In [10]:
pd.isnull(lines).sum()

Hindi     0
Telugu    0
dtype: int64

In [11]:
lines=lines[~pd.isnull(lines['Hindi'])]
lines.drop_duplicates(inplace=True)
lines=lines.sample(n=1000,random_state=42)
lines.shape

(1000, 2)

In [12]:
lines['Hindi']=lines['Hindi'].apply(lambda x: x.lower())
lines['Telugu']=lines['Telugu'].apply(lambda x: x.lower())
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub("'", '', x))
lines['Telugu']=lines['Telugu'].apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['Hindi']=lines['Hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Telugu']=lines['Telugu'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
remove_digits = str.maketrans('', '', digits)
lines['Hindi']=lines['Hindi'].apply(lambda x: x.translate(remove_digits))
lines['Telugu']=lines['Telugu'].apply(lambda x: x.translate(remove_digits))
lines['Hindi']=lines['Hindi'].apply(lambda x: x.strip())
lines['Telugu']=lines['Telugu'].apply(lambda x: x.strip())
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub(" +", " ", x))
lines['Telugu']=lines['Telugu'].apply(lambda x: re.sub(" +", " ", x))
lines['Telugu'] = lines['Telugu'].apply(lambda x : 'START_ '+ x + ' _END')
lines.head()

,Hindi,Telugu
1265,लेकिन अगर ये जाने वाले रास्ते पर अधिक है तब आप...,START_ ఇలాగా వెళ్ళేదారిలోనే బంధించేటట్లయితే మన...
862,यह साध्य होता है बिना उर्जा खपत का महा विस्फोट...,START_ మరియు ఇవి అన్నీ కూడా ఒక భారీగా శక్తి వి...
1986,हंसते हैं इसलिए मुझे निर्माणकर्ताओं के पैसा बन...,START_ నవ్వులు అందుకే నాకు డెవలపర్స్ డబ్బులు స...
18,वैसे राज की बात यह है कि तब तक सोने नहीं जाना ...,START_ ఆ రహస్యమేమిటంటే ఒక రోజులో రాయవలసిన పదాల...
2055,हालांकि सच्चाई कुछ अलग है,START_ కానీ అది నిజం కాదు _END


In [13]:
all_hindi_words=set()
for hindi in lines['Hindi']:
    for word in hindi.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)
all_telugu_words=set()
for telugu in lines['Telugu']:
    for word in telugu.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

In [15]:
print(len(all_hindi_words))
print(len(all_telugu_words))
lines['length_hindi_sentence']=lines['Hindi'].apply(lambda x:len(x.split(" ")))
lines['length_telugu_sentence']=lines['Telugu'].apply(lambda x:len(x.split(" ")))
lines.head()
lines[lines['length_hindi_sentence']>30].shape
lines=lines[lines['length_hindi_sentence']<=20]
lines=lines[lines['length_telugu_sentence']<=20]
lines.shape
print("maximum length of Hindi Sentence ",max(lines['length_hindi_sentence']))
print("maximum length of Telugu Sentence ",max(lines['length_telugu_sentence']))

4075
6225
maximum length of Hindi Sentence  20
maximum length of Telugu Sentence  20


In [16]:
len(all_hindi_words)
len(all_telugu_words)
lines['length_hindi_sentence']=lines['Hindi'].apply(lambda x:len(x.split(" ")))
lines['length_telugu_sentence']=lines['Telugu'].apply(lambda x:len(x.split(" ")))
lines.head()
lines[lines['length_hindi_sentence']>30].shape
lines=lines[lines['length_hindi_sentence']<=20]
lines=lines[lines['length_telugu_sentence']<=20]
lines.shape

(633, 4)

In [17]:
print("maximum length of Hindi Sentence ",max(lines['length_hindi_sentence']))
print("maximum length of Telugu Sentence ",max(lines['length_telugu_sentence']))
max_length_src=max(lines['length_hindi_sentence'])
max_length_tar=max(lines['length_telugu_sentence'])

maximum length of Hindi Sentence  20
maximum length of Telugu Sentence  20


In [18]:
input_words = sorted(list(all_hindi_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_hindi_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens
num_decoder_tokens += 1
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)
lines.head(10)

,Hindi,Telugu,length_hindi_sentence,length_telugu_sentence
2365,ये लोग ठगे हुए हैं विवश हैं,START_ వీరందరినీ మోసగించి బలవంతంగా నెట్టారు _END,7,6
1314,ये अन्य बहादुर पीढी़ है जो इस ग्रह पर लोकतंत्र...,START_ ఇది మరో హీరో ఆవిర్భావము ఈ భువికి ప్రజాస...,14,10
170,तो यह आपको दिखाता है कि हमारे पास ऐसे लोग हैं ...,START_ దాన్ని బట్టి మాదగ్గర అలాంటి పని చేయగల స...,17,10
2411,मैं अपने दायें कान से सुन नहीं सकती हूँ,START_ నా కుడి చెవితో ఏమి వినలేను _END,9,7
1371,आप माथ मत कीजिए,START_ దానిపై లెక్కలు అనవసరం _END,4,5
1952,और वे हमारे छोटेछोटे मित्रों के लिए रहने की जग...,START_ ఈ పైపులైన్లు మా చిన్న చిన్న మిత్రులకు ఆ...,13,10
908,जिससे मैं बहुती सुंदर कपड़ा बना सकने समर्थ हुई।...,START_ దానితోసాదారణ బట్ట లాంటి ఒక అందమైన వస్త్...,16,10
2409,मेरी चुनौती वो देहव्यापारी नहीं रहे हैं जो मुझ...,START_ ఈ సెక్స్ వ్యాపారం చేయిస్తూ నన్ను కొట్టే...,11,13
2366,उनमे से ९९९ प्रतिशत लोग वेश्यावृति को अपनाने म...,START_ పెర్సెంట్ వీరు వ్యభిచరించటానికి వ్యతిరే...,12,6
2348,मैं व्यावसायिक यौन शोषण के मुद्दे पर काम करती हूँ,START_ నేను కమర్షియల్ సెక్స్ ఎక్ష్ప్లైఠేషెన్ అ...,10,10


In [20]:
X, y = lines['Hindi'], lines['Telugu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [21]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [22]:
latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 300)            1222500   ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, None, 300)            1867800   ['input_4[0][0]']             
                                                                                            

In [25]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 20
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)
model.save_weights('nmt_weights.h5')

Epoch 1/20


C:\Users\bhavi\AppData\Local\Temp\ipykernel_2776\3760992297.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


7/7 [==============================] - 10s 666ms/step - loss: 8.7302 - val_loss: 8.7200
Epoch 2/20
7/7 [==============================] - 2s 303ms/step - loss: 8.7012 - val_loss: 8.6484
Epoch 3/20
7/7 [==============================] - 2s 298ms/step - loss: 8.2730 - val_loss: 7.9051
Epoch 4/20
7/7 [==============================] - 2s 300ms/step - loss: 7.6743 - val_loss: 7.6066
Epoch 5/20
7/7 [==============================] - 2s 303ms/step - loss: 7.3977 - val_loss: 7.7275
Epoch 6/20
7/7 [==============================] - 2s 310ms/step - loss: 7.2225 - val_loss: 7.5620
Epoch 7/20
7/7 [==============================] - 2s 312ms/step - loss: 7.1018 - val_loss: 7.7946
Epoch 8/20
7/7 [==============================] - 2s 363ms/step - loss: 7.0226 - val_loss: 7.6156
Epoch 9/20
7/7 [==============================] - 3s 378ms/step - loss: 6.9851 - val_loss: 7.9271
Epoch 10/20
7/7 [==============================] - 2s 337ms/step - loss: 6.9163 - val_loss: 7.6485
Epoch 11/20
7/7 [============

In [26]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

In [27]:
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [28]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: जिससे मैं बहुती सुंदर कपड़ा बना सकने समर्थ हुई। उसे मैं नियमित रूप से पहन सकू।
Actual Hindi Translation:  దానితోసాదారణ బట్ట లాంటి ఒక అందమైన వస్త్రాని ప్రింట్ చేయగలిగాను 
Predicted Hindi Translation:  ఈ ఈ ఈ ఈ ఒక 
1/1 [==============================] - 0s 19ms/step
Input English sentence: उन्हें आपकी सहानुभूति की जरूरत है
Actual Hindi Translation:  మీ సానుభూతి వారికి కావాలి ఏమ్పతి 
Predicted Hindi Translation:  ఈ 
